# Example usage

The poorly named `labtools` python package is a collection of diverse modules that might be of use to Staller Lab members and collaborators. 
- simple tools to shuffle/mutate sequences for library design
- tools to analyze raw sequencing data
- (hopefully) tools to run in-house neural net

## Sequence Design Tools

### Imports

In [1]:
from labtools import shuffle

### Shuffle a sequence

Create shuffles of the alphabet:

In [2]:
shuffles_list, names_list = shuffle.windowed_shuffle("ABCDEFGHIJKLMNOPQRSTUVWYXZ")
shuffles_list

['bcdaeFGHIJKLMNOPQRSTUVWYXZ',
 'AdfcbeGHIJKLMNOPQRSTUVWYXZ',
 'ABedfgcHIJKLMNOPQRSTUVWYXZ',
 'ABCgdhefIJKLMNOPQRSTUVWYXZ',
 'ABCDgehifJKLMNOPQRSTUVWYXZ',
 'ABCDEjigfhKLMNOPQRSTUVWYXZ',
 'ABCDEFihgjkLMNOPQRSTUVWYXZ',
 'ABCDEFGkilhjMNOPQRSTUVWYXZ',
 'ABCDEFGHlmkijNOPQRSTUVWYXZ',
 'ABCDEFGHInkljmOPQRSTUVWYXZ',
 'ABCDEFGHIJmloknPQRSTUVWYXZ',
 'ABCDEFGHIJKplmnoQRSTUVWYXZ',
 'ABCDEFGHIJKLnomqpRSTUVWYXZ',
 'ABCDEFGHIJKLMqoprnSTUVWYXZ',
 'ABCDEFGHIJKLMNosqprTUVWYXZ',
 'ABCDEFGHIJKLMNOqsrtpUVWYXZ',
 'ABCDEFGHIJKLMNOPtuqsrVWYXZ',
 'ABCDEFGHIJKLMNOPQutsrvWYXZ',
 'ABCDEFGHIJKLMNOPQRwstuvYXZ',
 'ABCDEFGHIJKLMNOPQRSywtvuXZ',
 'ABCDEFGHIJKLMNOPQRSTuvwxyZ',
 'ABCDEFGHIJKLMNOPQRSTUvzxwy']

Following code is used to style and truncate the pandas dataframe outputs for easier example viewing.

In [66]:
def format_index(s):
    return f"font-size: 8pt; max-width: 250px; text-overflow: ellipsis; overflow: hidden"
def format(s):
    return ["font-size: 10pt;"] *len(s)
def display_df(df, **kwargs):
    disp = df.head(10).style.applymap_index(format_index)
    disp = disp.applymap_index(format_index, axis = 0)
    disp = disp.apply(format)
    return disp

## Sequencing Analysis Tools

### Sort Processing Example
#### What you need
* fastq file for each sample in the sort
* bin counts (cells per bin) and bin values (mean or median fluorescence of the bin)
* **if you want perfect matches**: a csv with one column headered as "ArrayDNA" which contains your 120 bp AD sequences as DNA
* the sequence directly preceding your AD sequence in your reads (anchor sequence)
* if you have barcodes, you need the sequence directly preceding them and anteceding them (anchor sequences), otherwise ignore these. See the example with barcodes below.

#### What you get out
* a dataframe with your AD tiles as indices, normalized scores for tile at each bin, and the tile Activity value

**NOTE: the read cutoff is 10 reads total summed across all bins** This means that if a tile is not found at least 10 times combined across any of the fastq files, it will not be analysed. 

#### Use the sequence directly preceding the AD/tile as an anchor sequence. Additional characters between AD/tile and preceding anchor seqeunce will not work

My preceding anchor sequence is in blue while my AD/tile sequence is in green in this example read.

The anchor sequence preceding the barcode is purple while the anchor sequence anteceding is red. Barcode length is 11 by default. (Not necessary if you don't have barcodes)

##### Example read showing the default values. No need to change anything if your sequence matches the defaults.

<span style="font-size:10px">TCCCTGCGGGCTCTACTTCATCG<span style="color:blue;font-weight:800">GCTAGC</span><span style="color:green;font-weight:500;font-size:10px">**GGTTCTT...CTGCTAAA**</span>TGATAAATAGATGA<span style="color:purple;font-weight:800">GGGCCCG</span>TCAACATAGAA<span style="color:red;font-weight:800">GGAGAGAA</span>ACATCTAAAAAAGCGATA</span>

### Imports

In [25]:
from labtools.adtools import sort

### Initialize a sort

In [26]:
my_sort = sort.Sort(["../exampledata/bin1.fastq", "../exampledata/bin2.fastq", "../exampledata/bin3.fastq", 
                "../exampledata/bin4.fastq"], bin_counts = [100000,100000,100000,100000], 
               bin_values = [61,141,251,1462], design_file = "../exampledata/unique_seqs.csv")

### Get activity values

In [27]:
activities, total_reads, reads_per_bin = my_sort.process()

In [67]:
# activity + normalized abundance in each bin
display_df(activities)

,0,1,2,3,Activity
GAAGATCCAACTTGGTTTGATTCTGGTTCTCAATTTATCTTAAATTCTCAACAATTGGTTGAAGCTTTGTCTTTGTGTGATGATGCTGCTGGTTCTCAAGATAGAGAAGAGAATACTAAT,0.279148,0.293312,0.427540,0.000000,165.697515
GAAGCTTTGTCTTTGTGTGATGATTTGTTGGGTGATCAAGATAGAGAAGAGAATGATAATGATGGTGATTTGAAAGATAAACAACCATGTATTGCTGATTATGCTCATTTGGGTCCAGAA,0.110487,0.540278,0.325749,0.023487,199.019328
GATTTGGCTGAAGATGATGAAGTTATGTGTATGGAAGATGAAGTTCAATCTATTCAACCAAATCATGAAAGACCAGATGATGGTCCAGAATTGGAACATGGTTTGGAGAATGGTGCTAAA,0.376741,0.492582,0.130676,0.000000,125.235085
TCTGAATTGACTTCTACTTTGGGTATTTCTCATAGATTGCCACAATCTTTGACTCCATGTGTTAAGACTGGTTCTTTGCAATCTGGTGGTTTGGTTCAATCTGTTCCATTTGAAGAATTG,0.362058,0.279914,0.358029,0.000000,151.418558
AATCATGAAAGACCATCTTCTGGTCCAGAAGCTGAACATGGTTTGGAGAATGGTGCTAAAGAAATGGCTGATGATAAAGAAGAACAAGAGAAAGATAGAGATAATGAGAATCAAGGTGAA,0.611774,0.247237,0.093699,0.047290,164.835151
CCATCTGATTATGTTTCTGCTCATGAAGTTTCTTTGGCTGAAACTTCTGAAGTTATGTGTATGGAAGATGAAGTTCAATCTATTCAACCAAATCATGAAAGACCATCTTCTGGTCCAGAA,0.154808,0.367035,0.478157,0.000000,181.212555
GATGCTAAGAGAGATTTGGAAGAATGTCAGAAGATTGTTGCTGATCCATCTAATATTGAATTGGATGCTCCACCAGAAGCTAGATTGGCTCAAGCTGAATTTGGTTCTCAAGATTCTGCT,0.348924,0.504114,0.146962,0.000000,129.251907
GAAGGTAATTCTTCTAAAGCTGAAACTTGTCAACCACAACCAAGATCTAATAGAGAAATTGTTGTTGGTGCTTCTGTTTCTGATGTTTCTAGAGGTACTTTGATTACTCCTAAGAGAGAT,0.521596,0.324621,0.153782,0.000000,116.188376
CAAGACTTTATTTGTTCTGTTACTCAAGATGTTGGTATTAATTATACTCATCCACAGAATTTGCCAGGTGTTTCTAAAGATGGTACTTCTGTCTTCTTCTTTAATAAGACTGCTCATGCT,0.244171,0.458300,0.251391,0.046137,210.066902
TTTCATAAGACTGTTCAAGCTTATGGTACTGGTCAAAGGAAGAGAAGGAAGATTACTCCAACTTTGGTTAATGATGAACCAGTTAGATGGCATAAGACTGGTAGAACTAAACCAGTTATG,0.371347,0.407844,0.220809,0.000000,135.581225


In [8]:
# the total reads for each sequence
total_reads

GAAGATCCAACTTGGTTTGATTCTGGTTCTCAATTTATCTTAAATTCTCAACAATTGGTTGAAGCTTTGTCTTTGTGTGATGATGCTGCTGGTTCTCAAGATAGAGAAGAGAATACTAAT    44.0
GAAGCTTTGTCTTTGTGTGATGATTTGTTGGGTGATCAAGATAGAGAAGAGAATGATAATGATGGTGATTTGAAAGATAAACAACCATGTATTGCTGATTATGCTCATTTGGGTCCAGAA    21.0
GATTTGGCTGAAGATGATGAAGTTATGTGTATGGAAGATGAAGTTCAATCTATTCAACCAAATCATGAAAGACCAGATGATGGTCCAGAATTGGAACATGGTTTGGAGAATGGTGCTAAA    49.0
TCTGAATTGACTTCTACTTTGGGTATTTCTCATAGATTGCCACAATCTTTGACTCCATGTGTTAAGACTGGTTCTTTGCAATCTGGTGGTTTGGTTCAATCTGTTCCATTTGAAGAATTG    70.0
AATCATGAAAGACCATCTTCTGGTCCAGAAGCTGAACATGGTTTGGAGAATGGTGCTAAAGAAATGGCTGATGATAAAGAAGAACAAGAGAAAGATAGAGATAATGAGAATCAAGGTGAA    20.0
                                                                                                                            ... 
GACCCAACTGAATGGTTTGATTCTGGTGCTCAATTTATCTTGAATGCTCAACAATTGGTTGAAGCTCAATGTTTGGATGATAATTTGACTAGAGAATTGGAATCTAATGATGGTGCTTTG    23.0
TCTACTGATTCTACTCCAATGTTTGATTATGATAATTTGGAAGATAATTCTAAAGATTGGACTTCTTTGTTTGATAATGATATTCCAGTTACTACTG

In [68]:
# the reads of each tile per bin
display_df(reads_per_bin)

,0,1,2,3
GAAGATCCAACTTGGTTTGATTCTGGTTCTCAATTTATCTTAAATTCTCAACAATTGGTTGAAGCTTTGTCTTTGTGTGATGATGCTGCTGGTTCTCAAGATAGAGAAGAGAATACTAAT,11.000000,13.000000,20.000000,0.000000
GAAGCTTTGTCTTTGTGTGATGATTTGTTGGGTGATCAAGATAGAGAAGAGAATGATAATGATGGTGATTTGAAAGATAAACAACCATGTATTGCTGATTATGCTCATTTGGGTCCAGAA,2.000000,11.000000,7.000000,1.000000
GATTTGGCTGAAGATGATGAAGTTATGTGTATGGAAGATGAAGTTCAATCTATTCAACCAAATCATGAAAGACCAGATGATGGTCCAGAATTGGAACATGGTTTGGAGAATGGTGCTAAA,17.000000,25.000000,7.000000,0.000000
TCTGAATTGACTTCTACTTTGGGTATTTCTCATAGATTGCCACAATCTTTGACTCCATGTGTTAAGACTGGTTCTTTGCAATCTGGTGGTTTGGTTCAATCTGTTCCATTTGAAGAATTG,23.000000,20.000000,27.000000,0.000000
AATCATGAAAGACCATCTTCTGGTCCAGAAGCTGAACATGGTTTGGAGAATGGTGCTAAAGAAATGGCTGATGATAAAGAAGAACAAGAGAAAGATAGAGATAATGAGAATCAAGGTGAA,11.000000,5.000000,2.000000,2.000000
CCATCTGATTATGTTTCTGCTCATGAAGTTTCTTTGGCTGAAACTTCTGAAGTTATGTGTATGGAAGATGAAGTTCAATCTATTCAACCAAATCATGAAAGACCATCTTCTGGTCCAGAA,3.000000,8.000000,11.000000,0.000000
GATGCTAAGAGAGATTTGGAAGAATGTCAGAAGATTGTTGCTGATCCATCTAATATTGAATTGGATGCTCCACCAGAAGCTAGATTGGCTCAAGCTGAATTTGGTTCTCAAGATTCTGCT,8.000000,13.000000,4.000000,0.000000
GAAGGTAATTCTTCTAAAGCTGAAACTTGTCAACCACAACCAAGATCTAATAGAGAAATTGTTGTTGGTGCTTCTGTTTCTGATGTTTCTAGAGGTACTTTGATTACTCCTAAGAGAGAT,20.000000,14.000000,7.000000,0.000000
CAAGACTTTATTTGTTCTGTTACTCAAGATGTTGGTATTAATTATACTCATCCACAGAATTTGCCAGGTGTTTCTAAAGATGGTACTTCTGTCTTCTTCTTTAATAAGACTGCTCATGCT,9.000000,19.000000,11.000000,4.000000
TTTCATAAGACTGTTCAAGCTTATGGTACTGGTCAAAGGAAGAGAAGGAAGATTACTCCAACTTTGGTTAATGATGAACCAGTTAGATGGCATAAGACTGGTAGAACTAAACCAGTTATG,17.000000,21.000000,12.000000,0.000000


#### Locate any tiles, not just perfect matches
Simply do not include the design file. A short (less than 120 bp) tile will probably appear with a significant number of reads. This is **probably** your untransformed background (cells transformed with a plasmid that did not get a tile). Most of the non-perfect matching tiles will be sequencing errors. You might find a way to map these back to their true tile. A few of the non-perfect matching tiles (likely those with a significant number of reads) will be sequencing library PCR amplification errors. An even fewer number of the non-perfect matching reads could be mutants that arose within the actual cell, or free tiles. Someone may put in the effort to distinguish these categories in the future.

In [10]:
my_sort = sort.Sort(["../exampledata/bin1.fastq", "../exampledata/bin2.fastq", "../exampledata/bin3.fastq", 
                "../exampledata/bin4.fastq"], bin_counts = [100000,100000,100000,100000], 
               bin_values = [61,141,251,1462])
activities_no_design, numreads_total, _ = my_sort.process()

In [11]:
display_df(activities_no_design)

,0,1,2,3,Activity
GAAGATCCAACTTGGTTTGATTCTGGTTCTCAATTTATCTTAAATTCTCAACAATTGGTTGAAGCTTTGTCTTTGTGTGATGATGCTGCTGGTTCTCAAGATAGAGAAGAGAATACTAAT,0.269721,0.290977,0.439302,0.000000,167.745576
GAAGCTTTGTCTTTGTGTGATGATTTGTTGGGTGATCAAGATAGAGAAGAGAATGATAATGATGGTGATTTGAAAGATAAACAACCATGTATTGCTGATTATGCTCATTTGGGTCCAGAA,0.106476,0.534574,0.333835,0.025115,202.380348
GATTTGGCTGAAGATGATGAAGTTATGTGTATGGAAGATGAAGTTCAATCTATTCAACCAAATCATGAAAGACCAGATGATGGTCCAGAATTGGAACATGGTTTGGAGAATGGTGCTAAA,0.368831,0.495122,0.136047,0.000000,126.458644
GGTTAATTAAGGCGCGCCACTTCTAAATAAGCGA,0.224803,0.533542,0.241655,0.000000,149.597758
TCTGAATTGACTTCTACTTTGGGTATTTCTCATAGATTGCCACAATCTTTGACTCCATGTGTTAAGACTGGTTCTTTGCAATCTGGTGGTTTGGTTCAATCTGTTCCATTTGAAGAATTG,0.351449,0.278970,0.369581,0.000000,153.537999
AATCATGAAAGACCATCTTCTGGTCCAGAAGCTGAACATGGTTTGGAGAATGGTGCTAAAGAAATGGCTGATGATAAAGAAGAACAAGAGAAAGATAGAGATAATGAGAATCAAGGTGAA,0.601117,0.249419,0.097906,0.051559,171.789477
CCATCTGATTATGTTTCTGCTCATGAAGTTTCTTTGGCTGAAACTTCTGAAGTTATGTGTATGGAAGATGAAGTTCAATCTATTCAACCAAATCATGAAAGACCATCTTCTGGTCCAGAA,0.148835,0.362300,0.488865,0.000000,182.868328
GATGCTAAGAGAGATTTGGAAGAATGTCAGAAGATTGTTGCTGATCCATCTAATATTGAATTGGATGCTCCACCAGAAGCTAGATTGGCTCAAGCTGAATTTGGTTCTCAAGATTCTGCT,0.341150,0.506048,0.152801,0.000000,130.516131
GAAGGTAATTCTTCTAAAGCTGAAACTTGTCAACCACAACCAAGATCTAATAGAGAAATTGTTGTTGGTGCTTCTGTTTCTGATGTTTCTAGAGGTACTTTGATTACTCCTAAGAGAGAT,0.512160,0.327263,0.160578,0.000000,117.690761
CAAGACTTTATTTGTTCTGTTACTCAAGATGTTGGTATTAATTATACTCATCCACAGAATTTGCCAGGTGTTTCTAAAGATGGTACTTCTGTCTTCTTCTTTAATAAGACTGCTCATGCT,0.236315,0.455403,0.258734,0.049547,216.006969


In [12]:
numreads_total

GAAGATCCAACTTGGTTTGATTCTGGTTCTCAATTTATCTTAAATTCTCAACAATTGGTTGAAGCTTTGTCTTTGTGTGATGATGCTGCTGGTTCTCAAGATAGAGAAGAGAATACTAAT    44.0
GAAGCTTTGTCTTTGTGTGATGATTTGTTGGGTGATCAAGATAGAGAAGAGAATGATAATGATGGTGATTTGAAAGATAAACAACCATGTATTGCTGATTATGCTCATTTGGGTCCAGAA    21.0
GATTTGGCTGAAGATGATGAAGTTATGTGTATGGAAGATGAAGTTCAATCTATTCAACCAAATCATGAAAGACCAGATGATGGTCCAGAATTGGAACATGGTTTGGAGAATGGTGCTAAA    49.0
GGTTAATTAAGGCGCGCCACTTCTAAATAAGCGA                                                                                          24.0
TCTGAATTGACTTCTACTTTGGGTATTTCTCATAGATTGCCACAATCTTTGACTCCATGTGTTAAGACTGGTTCTTTGCAATCTGGTGGTTTGGTTCAATCTGTTCCATTTGAAGAATTG    70.0
                                                                                                                            ... 
TCTACTGATTCTACTCCAATGTTTGATTATGATAATTTGGAAGATAATTCTAAAGATTGGACTTCTTTGTTTGATAATGATATTCCAGTTACTACTGATGATGTTTCTTTGGCTGATAAA    24.0
TCTACTGGTCAAGTCTTGTTTGATATTGATGACTTTAGATGGTTGTTGGATCCAGATGATGAACAATTGGGTAAAGAAGCTATCTTGTCTGATCAAT

### Get data for reads that include barcodes AND tiles
Support for barcoded only data may or may not be added in the future.

In [13]:
activities_barcoded, total_reads, reads_per_bin = my_sort.process(barcoded = True)

Note that you get back less tiles using this method. The reason for this is because the read must have a locatable barcode AND tile, which is less likely than having one or the other. The primary reason for doing this analysis is assessing per-transformant variation. The assumption is that unique tile-barcode pairs come from unique original transformants.

In [14]:
display_df(activities_barcoded)

,,0,1,2,3,Activity
GAAGATCCAACTTGGTTTGATTCTGGTTCTCAATTTATCTTAAATTCTCAACAATTGGTTGAAGCTTTGTCTTTGTGTGATGATGCTGCTGGTTCTCAAGATAGAGAAGAGAATACTAAT,ATCGTTACGTC,0.289081,0.312643,0.398276,0.000000,161.683873
GATTTGGCTGAAGATGATGAAGTTATGTGTATGGAAGATGAAGTTCAATCTATTCAACCAAATCATGAAAGACCAGATGATGGTCCAGAATTGGAACATGGTTTGGAGAATGGTGCTAAA,GGAAGGTAGTA,0.234149,0.557113,0.208738,0.000000,145.229204
TCTGAATTGACTTCTACTTTGGGTATTTCTCATAGATTGCCACAATCTTTGACTCCATGTGTTAAGACTGGTTCTTTGCAATCTGGTGGTTTGGTTCAATCTGTTCCATTTGAAGAATTG,GAGACCAAATC,0.331376,0.288808,0.379816,0.000000,156.269710
GATGCTAAGAGAGATTTGGAAGAATGTCAGAAGATTGTTGCTGATCCATCTAATATTGAATTGGATGCTCCACCAGAAGCTAGATTGGCTCAAGCTGAATTTGGTTCTCAAGATTCTGCT,CAGGAAAATCG,0.305253,0.558685,0.136062,0.000000,131.546634
CAAGACTTTATTTGTTCTGTTACTCAAGATGTTGGTATTAATTATACTCATCCACAGAATTTGCCAGGTGTTTCTAAAGATGGTACTTCTGTCTTCTTCTTTAATAAGACTGCTCATGCT,ACATGAACTTA,0.244871,0.416160,0.280666,0.058303,229.301369
TTTCATAAGACTGTTCAAGCTTATGGTACTGGTCAAAGGAAGAGAAGGAAGATTACTCCAACTTTGGTTAATGATGAACCAGTTAGATGGCATAAGACTGGTAGAACTAAACCAGTTATG,ACACTTTAGGG,0.362356,0.409622,0.228022,0.000000,137.093891
CAGAAGATTGTTGCTGATCCATCTAATATTGAAGCTGATACTCCACCAGAAGCTAGAGCTGCTCAAGCTGAAGCTGGTGCTCAAGATTCTGCTGCTGCTGCTGGTACTGGTAAGACTAAT,ACTGGTTAGTT,0.436420,0.384016,0.179565,0.000000,125.838509
GCTGGTGCTTTGAAAGATAAACAACCATGTATTGCTGACTTTGCTCATTTGGGTCCAGAAGACTTTAAGAGAGATTTGGAAGAATGTCAGAAGATTGTCTTGGATCCAGCTAATATTGAA,TCCGCTAGCCT,0.215940,0.543430,0.240631,0.000000,150.194183
CATTTGGGTATTGAAGAAGATGAGAAAGAAGGTGATGAAGTTGTTGATAAGATCTTTGAACAACAACCACAACAATTGGTTGTTAAGAGAGGTGATAAAGCTGAACAAGAAGTTGATGAA,AAAGCGTACCT,0.572517,0.277370,0.150113,0.000000,111.711040
GAAGGTAATTCTTCTAAAGCTGAAACTTGTCAACCACAACCAAGATCTAATAGAGAAATTGTTGTTGGTGCTTCTGTTTCTGATGTTTCTAGAGGTACTTTGATTACTCCTAAGAGAGAT,TTGTCCCTAAG,0.614218,0.259423,0.126360,0.000000,105.762145


### Support for custom anchor sequences
If your tiles or barcodes have a custom anchor sequence (AKA the non-variable portion of the read that is used to locate the variable portion of the read), you can specify that in the kwargs of your Sort(). This passes the arguments to the pull_A() function used on each read to locate the sequence of interest (AKA AD or tile). 

#### Use the sequence directly preceding the AD/tile as an anchor sequence. Additional characters between AD/tile and preceding anchor seqeunce will not work

My preceding anchor sequence is in blue while my AD/tile sequence is in green in this example read.

The anchor sequence preceding the barcode is purple while the anchor sequence anteceding is red. Barcode length is 11 by default. (Not necessary if you don't have barcodes)

##### Example read

<span style="font-size:10px">TCCCTGCGGGCTCTACTTCATCG<span style="color:blue;font-weight:800">GCTAGC</span><span style="color:green;font-weight:500;font-size:10px">**GGTTCTT...CTGCTAAA**</span>TGATAAATAGATGA<span style="color:purple;font-weight:800">GGGCCCG</span>TCAACATAGAA<span style="color:red;font-weight:800">GGAGAGAA</span>ACATCTAAAAAAGCGATA</span>

#### Specify alternate values in a dictionary and pass that dictionary to Sort.process()

In [15]:
# these are the default values which would work for my example read above
# no input is required if they work for you
kwargs = {"ad_preceder":"GCTAGC", "bc_preceder":"GGGCCCG", "bc_anteceder":"GGAGAGAA", "bclength":11, "ad_length":120}

In [16]:
activities_barcoded, _, _ = my_sort.process(barcoded = True, **kwargs)

In [17]:
import pandas as pd

In [18]:
pd.set_option('display.max_colwidth', 1)

#### Example output for pull_AD
This might be useful for someone who wants to use pull_AD to analyze reads for a purpose outside of calculating activities. 

In [19]:
from labtools.adtools.finder import pull_AD

In [20]:
read = "TCCCTGCGGGCTCTACTTCATCGGCTAGCGGTTCTTCTAAATTGAGATGTGATAATAATGCTGCTGCTCATGTTAAATTGGATTCATTTCCAGCTGGTGTTAGATTTGATACATCTGATGAAGAATTGTTGGAACATTTGGCTGCTAAATGATAAATAGATGAGGGCCCGTCAACATAGAAGGAGAGAAACATCTAAAAAAGCGATA"
pull_AD(read, kwargs)

('GGTTCTTCTAAATTGAGATGTGATAATAATGCTGCTGCTCATGTTAAATTGGATTCATTTCCAGCTGGTGTTAGATTTGATACATCTGATGAAGAATTGTTGGAACATTTGGCTGCTAAA',
 'TCAACATAGAA')

### Counting tiles in a fastq file
This essentially applies the pull_AD function shown above to every read in the fastq file. You can also pass the kwargs dict to it to specify custom anchor sequences. 

In [21]:
from labtools.adtools.pair_counter import seq_counter
seq_counter("../exampledata/mini.fastq")

GGTTCTTCTAAATTGAGATGTGATAATAATGCTGCTGCTCATGTTAAATTGGATTCATTTCCAGCTGGTGTTAGATTTGATACATCTGATGAAGAATTGTTGGAACATTTGGCTGCTAAA    1
GAAGAATTGTTTTTACATTTGTCTGCTAAGATTGGTAGATCTTCTAGGAAACCACATCCATTCTTGGATGAATTTATTCATACTTTGGTTGAAGAAGATGGTATTTGTAGAACTCATCCA    3
dtype: int64

- Use barcoded = True  to count tile and barcode pairs
- Use the design_file flag to look for specific tiles

In [22]:
counts = seq_counter("../exampledata/bin1.fastq", barcoded = True, design_to_use="../exampledata/unique_seqs.csv")
counts

GAAGATCCAACTTGGTTTGATTCTGGTTCTCAATTTATCTTAAATTCTCAACAATTGGTTGAAGCTTTGTCTTTGTGTGATGATGCTGCTGGTTCTCAAGATAGAGAAGAGAATACTAAT  ATCGTTACGTC    11.0
GAAGCTTTGTCTTTGTGTGATGATTTGTTGGGTGATCAAGATAGAGAAGAGAATGATAATGATGGTGATTTGAAAGATAAACAACCATGTATTGCTGATTATGCTCATTTGGGTCCAGAA  ACCATTTAGCG    1.0 
GATTTGGCTGAAGATGATGAAGTTATGTGTATGGAAGATGAAGTTCAATCTATTCAACCAAATCATGAAAGACCAGATGATGGTCCAGAATTGGAACATGGTTTGGAGAATGGTGCTAAA  GGAAGGTAGTA    5.0 
GGTCAAAGGAAGAGAAGGAAGATTACTCCAACTTTGGTTAATGATGAACCAGTTAGATGGCATAAGACTGGTAGAACTAAACCAGTTATGTTGTCTGGTGTTCAAAGAGGTTGTAAGAAA  TAGCTCAAACC    4.0 
TCTGAATTGACTTCTACTTTGGGTATTTCTCATAGATTGCCACAATCTTTGACTCCATGTGTTAAGACTGGTTCTTTGCAATCTGGTGGTTTGGTTCAATCTGTTCCATTTGAAGAATTG  GAGACCAAATC    21.0
                                                                                                                                         ... 
CAAGCTATTGCTAATGATCAACAAGCTGTTGAAGCTGCTTCTGCTTGTGATGATGCTGCTGGTTCTCAAGATAGAGAAGAGAATACTAATTCTGGTTCTGCTAAAGATAAACAACCATGT  TCGTCTGACAC    1.0 
AAAGAT

## Predictors (coming soon?)